In [1]:
import pandas as pd
import numpy as np
import flow

In [2]:
df = flow.get_irrigation_depth_pressure_data()
df.head()

,STATE,Average Well Depth (ft),Average operating pressure (psi),Average pumping capacity (gpm),Elec_Total_Acres,NG_Total_Acres,Propane_Total_Acres,Diesel_Total_Acres,Gas_Total_Acres
0,AL,177,49,302.0,49864,180,0,55664,373
1,AK,80,46,92.0,0,0,0,0,10
2,AZ,392,35,732.0,484696,22620,0,61001,3774
3,AR,128,27,1362.0,2585513,112821,51919,2948504,42723
4,CA,354,38,680.0,5989989,48304,48514,1349858,32
